# Introduction à la programmation linéaire avec Python - Partie 4
## Exemples du monde réel - Problème de mélange

Article écrit par  : Alex Keen https://github.com/benalexkeen/Introduction-to-linear-programming

On va faire des saucisses!

Nous avons les ingrédients suivants à notre disposition:


| Ingrédient | Cout (€/kg)  | Disponibilité (kg)|
|------------|--------------|-------------------|
|  Porc      | 4.32         | 30                |
|  Viande    | 2.46         | 20                |
|  Amidon    | 1.86         | 17                |

ON va faire 2 types de saucisses:
* Economie(>40% Porc)
* Premium (>60% Porc)

Une saucisse pèse 50 g ( soit 0.05 kg)

Selon les réglementations gouvernementales, la plus grande quantité d'amidon que nous pouvons utiliser dans nos saucisses est de 25%

Nous avons un contrat avec un boucher, et avons déjà acheté 23 kg de porc, qui va de gater s'il n'est pas utilisé.

Nous avons une demande pour 350 saucisses économiques et 500 saucisses de première qualité.

Nous devons trouver la manière la plus rentable de mélanger nos saucisses.


Modélisons le problème :

  *p<sub>e</sub>* = Porc dans la saucisse économique (kg)  
  *w<sub>e</sub>* = Viande dans la saucisse économique (kg)  
  *s<sub>e</sub>* = Amidon dans la saucisse économique (kg)  
  *p<sub>p</sub>* = Pork dans la saucisse premium (kg)  
  *w<sub>p</sub>* = Viande dans la saucisse premium  (kg)  
  *s<sub>p</sub>* = Amidon dans la saucisse premium  (kg)  

Nous voulons minimiser les couts de cette façon : Note Nico : Vérifier d'ou sort 0.72 0.41 et 0.31 ?

Cout = 0.72(*p<sub>e</sub>* + *p<sub>p</sub>*) + 0.41(*w<sub>e</sub>* + *w<sub>p</sub>*) + 0.31(*s<sub>e</sub>* + *s<sub>p</sub>*)


Avec les contraintes suivantes :

  *p<sub>e</sub>* + *w<sub>e</sub>* + *s<sub>e</sub>* = 350 \* 0.05  
  *p<sub>p</sub>* + *w<sub>p</sub>* + *s<sub>p</sub>* = 500 \* 0.05  
  *p<sub>e</sub>* ≥ 0.4(*p<sub>e</sub>* + *w<sub>e</sub>* + *s<sub>e</sub>*)  
  *p<sub>p</sub>* ≥ 0.6(*p<sub>p</sub>* + *w<sub>p</sub>* + *s<sub>p</sub>*)  
  *s<sub>e</sub>* ≤ 0.25(*p<sub>e</sub>* + *w<sub>e</sub>* + *s<sub>e</sub>*)  
  *s<sub>p</sub>* ≤ 0.25(*p<sub>p</sub>* + *w<sub>p</sub>* + *s<sub>p</sub>*)  
  *p<sub>e</sub>* + *p<sub>p</sub>* ≤ 30  
  *w<sub>e</sub>* + *w<sub>p</sub>* ≤ 20  
  *s<sub>e</sub>* + *s<sub>p</sub>* ≤ 17  
  *p<sub>e</sub>* + *p<sub>p</sub>* ≥ 23

In [1]:
import pulp

In [2]:
# Instantier la classe de notre problème :
model = pulp.LpProblem("Cost minimising blending problem", pulp.LpMinimize)

Ici, nous avons 6 variables de décision, nous pourrions les nommer individuellement, mais cela n'augmenterait pas si nous avions des centaines / milliers de variables (vous ne voulez pas les saisir toutes à la main plusieurs fois).

Nous allons créer quelques listes à partir desquelles nous pouvons créer des index de tuple.

In [3]:
# On construit nos variables de décision
sausage_types = ['economy', 'premium']
ingredients = ['pork', 'wheat', 'starch']

Chacune de ces variables de décision aura des caractéristiques similaires (borne inférieure de 0, variables continues). Par conséquent, nous pouvons utiliser la fonctionnalité dict de l'objet LpVariable de PuLP, nous pouvons fournir nos index de tuple.

Ces tuples seront des clés pour le dict ing_weight des variables de décision

In [4]:
ing_weight = pulp.LpVariable.dicts("weight kg",
                                     ((i, j) for i in sausage_types for j in ingredients),
                                     lowBound=0,
                                     cat='Continuous')

PuLP fournit un calcul vectoriel lpSum pour la somme d'une liste d'expressions linéaires.

Alors que nous n'avons que 6 variables de décision, je montrerai comment le problème serait construit d'une manière qui pourrait être étendue à de nombreuses variables en utilisant des compréhensions de liste.

In [5]:
# Objective Function
model += (
    pulp.lpSum([
        4.32 * ing_weight[(i, 'pork')]
        + 2.46 * ing_weight[(i, 'wheat')]
        + 1.86 * ing_weight[(i, 'starch')]
        for i in sausage_types])
)

Maintenant, nous ajoutons nos contraintes, gardons à nouveau à l'esprit ici comment l'utilisation de la compréhension de liste permet de passer à de nombreux ingrédients ou types de saucisses

In [6]:
# Constraints
# 350 economy and 500 premium sausages at 0.05 kg
model += pulp.lpSum([ing_weight['economy', j] for j in ingredients]) == 350 * 0.05
model += pulp.lpSum([ing_weight['premium', j] for j in ingredients]) == 500 * 0.05

# Economy has >= 40% pork, premium >= 60% pork
model += ing_weight['economy', 'pork'] >= (
    0.4 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))

model += ing_weight['premium', 'pork'] >= (
    0.6 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

# Sausages must be <= 25% starch
model += ing_weight['economy', 'starch'] <= (
    0.25 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))

model += ing_weight['premium', 'starch'] <= (
    0.25 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

# We have at most 30 kg of pork, 20 kg of wheat and 17 kg of starch available
model += pulp.lpSum([ing_weight[i, 'pork'] for i in sausage_types]) <= 30
model += pulp.lpSum([ing_weight[i, 'wheat'] for i in sausage_types]) <= 20
model += pulp.lpSum([ing_weight[i, 'starch'] for i in sausage_types]) <= 17

# We have at least 23 kg of pork to use up
model += pulp.lpSum([ing_weight[i, 'pork'] for i in sausage_types]) >= 23

In [7]:
# Solve our problem
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [8]:
for var in ing_weight:
    var_value = ing_weight[var].varValue
    print "The weight of {0} in {1} sausages is {2} kg".format(var[1], var[0], var_value)

The weight of starch in premium sausages is 6.25 kg
The weight of starch in economy sausages is 4.375 kg
The weight of wheat in economy sausages is 6.125 kg
The weight of wheat in premium sausages is 2.75 kg
The weight of pork in economy sausages is 7.0 kg
The weight of pork in premium sausages is 16.0 kg


In [9]:
total_cost = pulp.value(model.objective)

print "The total cost is €{} for 350 economy sausages and 500 premium sausages".format(round(total_cost, 2))

The total cost is €140.96 for 350 economy sausages and 500 premium sausages
